In [1]:
from dotenv import load_dotenv
load_dotenv()

import operator

In [2]:
import operator
import logging
import os
from typing import Annotated, Literal, TypedDict, Sequence
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, FunctionMessage
from langchain_core.tools import BaseTool, Tool
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.language_models import LanguageModelLike
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langgraph.checkpoint import MemorySaver
from utils.prompt_loader import load_markdown_prompt
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain_community.tools.wolfram_alpha import WolframAlphaQueryRun
from langgraph.prebuilt.tool_executor import ToolInvocation

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define the state
class MetaPromptingState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]
    error_log: list[str]
    turn_count: int

# Load meta-prompter instructions
META_PROMPTER_INSTRUCTIONS = load_markdown_prompt("../prompts/meta-prompter.md")

MAX_TURNS = 15  # Increased maximum number of turns for more complex queries

def create_meta_prompting_agent(
    model: LanguageModelLike,
    tools: Sequence[BaseTool],
    checkpointer: MemorySaver = None,
):
    # Create ToolExecutor
    tool_executor = ToolExecutor(tools)

    # Create the meta-prompter node
    def meta_prompter(state: MetaPromptingState, config: RunnableConfig):
        messages = state['messages']
        turn_count = state.get('turn_count', 0) + 1
        
        if turn_count > MAX_TURNS:
            logger.info(f"Reached maximum turns ({MAX_TURNS}). Forcing end of conversation.")
            return {
                "messages": [AIMessage(content="I apologize, but I've been unable to provide a satisfactory answer within a reasonable number of steps. Here's my best attempt at a final answer based on what we've discussed: [Summary of the conversation]")],
                "turn_count": turn_count,
                "error_log": state.get("error_log", []) + ["Reached maximum turns"],
            }
        
        response = model.invoke(
            [
                HumanMessage(content=f"{META_PROMPTER_INSTRUCTIONS}\n\nRemember to use available tools for up-to-date information when necessary. When you have a final answer, start your response with 'FINAL ANSWER:' and be sure it's comprehensive."),
                *messages
            ],
            config
        )
        
        result = {
            "messages": [AIMessage(content=f"Meta-Prompter: {response.content}")],
            "turn_count": turn_count,
            "error_log": state.get("error_log", []),
        }
        
        return result

    # Create the expert node with ReAct-like behavior
    def expert_node(state: MetaPromptingState, config: RunnableConfig):
        messages = state['messages']
        tool_names = ", ".join([tool.name for tool in tools])
        
        expert_prompt = f"""You are an expert assistant with access to the following tools: {tool_names}. 
        Use them when necessary to provide accurate and up-to-date information. 
        To use a tool, respond with the tool name and input in the following format:
        Tool: <tool_name>
        Input: <tool_input>
        
        If you have a final answer, start your response with 'FINAL ANSWER:' and ensure it's comprehensive.
        
        Current conversation:
        {messages}
        
        What would you like to do next? Consider using a tool if you need current information or specific data."""
        
        response = model.invoke([HumanMessage(content=expert_prompt)], config)
        
        if "Tool:" in response.content and "Input:" in response.content:
            tool_name = response.content.split("Tool:")[1].split("\n")[0].strip()
            tool_input = response.content.split("Input:")[1].strip()
            
            logger.info(f"Using tool: {tool_name}")
            
            try:
                tool_invocation = ToolInvocation(tool=tool_name, tool_input=tool_input)
                tool_result = tool_executor.invoke(tool_invocation)
                
                result = {
                    "messages": [
                        AIMessage(content=response.content),
                        FunctionMessage(content=str(tool_result), name=tool_name)
                    ],
                    "error_log": state.get("error_log", [])
                }
            except Exception as e:
                error_message = f"Error executing {tool_name}: {str(e)}"
                logger.error(error_message)
                result = {
                    "messages": [AIMessage(content=f"I encountered an error while trying to use the {tool_name} tool. I'll try a different approach.")],
                    "error_log": state.get("error_log", []) + [error_message]
                }
        else:
            result = {
                "messages": [AIMessage(content=response.content)],
                "error_log": state.get("error_log", [])
            }
        
        result["turn_count"] = state.get("turn_count", 0)
        return result

    # Define the function to determine whether to continue or end
    def should_continue(state: MetaPromptingState) -> Literal["continue", "expert", "end"]:
        last_message = state['messages'][-1].content if state['messages'] else ""
        
        if "FINAL ANSWER:" in last_message:
            logger.info("FINAL ANSWER detected, ending conversation")
            return "end"
        elif state.get("turn_count", 0) >= MAX_TURNS:
            logger.info("Maximum turns reached, ending conversation")
            return "end"
        elif last_message.startswith("Meta-Prompter:"):
            return "expert"
        else:
            return "continue"

    # Create the graph
    workflow = StateGraph(MetaPromptingState)

    # Add nodes
    workflow.add_node("meta_prompter", meta_prompter)
    workflow.add_node("expert", expert_node)

    # Set entry point
    workflow.set_entry_point("meta_prompter")

    # Add edges
    workflow.add_conditional_edges(
        "meta_prompter",
        should_continue,
        {
            "continue": "meta_prompter",
            "expert": "expert",
            "end": END
        }
    )
    workflow.add_conditional_edges(
        "expert",
        should_continue,
        {
            "continue": "meta_prompter",
            "expert": "expert",
            "end": END
        }
    )

    # Compile the graph
    return workflow.compile(checkpointer=checkpointer)

# Example usage
if __name__ == "__main__":
    # Create the language model
    model = ChatOpenAI(model_name="gpt-4", temperature=0)

    # Create instances of the tools
    tools = [
        TavilySearchResults(max_results=3),
    ]

    # Try to add Wolfram Alpha tool if available
    wolfram_alpha_appid = os.getenv("WOLFRAM_ALPHA_APPID")
    if wolfram_alpha_appid:
        try:
            wolfram = WolframAlphaAPIWrapper()
            tools.append(WolframAlphaQueryRun(api_wrapper=wolfram))
            logger.info("Wolfram Alpha tool added successfully.")
        except Exception as e:
            logger.warning(f"Failed to initialize Wolfram Alpha tool: {str(e)}")
    else:
        logger.warning("WOLFRAM_ALPHA_APPID not set. Skipping Wolfram Alpha tool.")

    # Try to add Wikipedia tool if available
    try:
        from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
        wikipedia = WikipediaAPIWrapper()
        tools.append(Tool(
            name="Wikipedia",
            func=wikipedia.run,
            description="Useful for querying Wikipedia to get information on a wide range of topics."
        ))
        logger.info("Wikipedia tool added successfully.")
    except ImportError:
        logger.warning("Wikipedia package not found. Proceeding without Wikipedia tool.")
    
    agent = create_meta_prompting_agent(model, tools)
    
    # Use streaming with improved output formatting
    try:
        complex_query = """
        Compare the economic impact of renewable energy adoption in Germany and China over the last decade. 
        Include data on their current energy mix, major renewable projects, and how this shift has affected 
        their carbon emissions and job markets. Also, provide a brief forecast of their renewable energy goals 
        for the next 5 years.
        """
        for chunk in agent.stream({"messages": [HumanMessage(content=complex_query)], "error_log": [], "turn_count": 0}):
            for key, value in chunk.items():
                if 'messages' in value:
                    for message in value['messages']:
                        print(f"\n{'=' * 40}")
                        print(f"{message.type.capitalize()}:")
                        print(f"{'=' * 40}")
                        print(message.content)
                elif key == 'error_log':
                    print(f"\n{'=' * 40}")
                    print("Error Log:")
                    print(f"{'=' * 40}")
                    for error in value:
                        print(error)
                else:
                    print(f"\n{'=' * 40}")
                    print(f"{key.capitalize()}:")
                    print(f"{'=' * 40}")
                    for sub_key, sub_value in value.items():
                        print(f"{sub_key}: {sub_value}")
            print("\n" + "-" * 80 + "\n")
    except Exception as e:
        logger.exception("An error occurred during execution")
        raise

2024-07-16 22:43:18,870 - INFO - Wolfram Alpha tool added successfully.
2024-07-16 22:43:18,910 - INFO - Wikipedia tool added successfully.
2024-07-16 22:43:29,176 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Ai:
Meta-Prompter: This is a complex question that requires research, data analysis, and writing skills. Here's a plan:

1. Research Expert: Gather data on the energy mix, major renewable projects, carbon emissions, job markets, and renewable energy goals for the next 5 years in Germany and China.
2. Data Analysis Expert: Analyze the data to compare the economic impact of renewable energy adoption in both countries.
3. Writing Expert: Write a comprehensive comparison based on the analysis.

Let's start with the Research Expert. 

Research Expert:
"""
Please gather data on the following for both Germany and China over the last decade:

1. Energy mix: What percentage of their energy comes from renewable sources? What are the main types of renewable energy they use?
2. Major renewable projects: What are some of the major renewable energy projects they have undertaken?
3. Carbon emissions: How have their carbon emissions changed with the adoption of renewable energy?
4. Job markets: How h

2024-07-16 22:43:30,658 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-16 22:43:30,661 - INFO - Using tool: tavily_search_results_json



Ai:
Tool: tavily_search_results_json
Input: {"search_query": "Germany renewable energy statistics 2021"}

Function:
[]

--------------------------------------------------------------------------------



2024-07-16 22:43:34,531 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...


Ai:
Tool: Wikipedia
Input: "Renewable energy in China"

Function:
Page: Renewable energy in China
Summary: China is the world's leader in electricity production from renewable energy sources, with over triple the generation of the second-ranking country, the United States. China's renewable energy sector is growing faster than its fossil fuels and nuclear power capacity, and is expected to contribute 43% of global renewable capacity growth. China's total renewable energy capacity exceeded 1,000 GW in 2021, accounting for 43.5 per cent of the country's total power generation capacity, 10.2 percentage points higher than in 2015. The country aims to have 80 per cent of its total energy mix come from non-fossil fuel sources by 2060, and achieve a combined 1,200 GW of solar and wind capacity by 2030. In 2023, it was reported that China was on track to reach 1,371 gigawatts of wind and solar by 2025, five years ahead of target due to new renewables installations breaking records.
Although C

2024-07-16 22:44:10,722 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Ai:
Meta-Prompter: Meta-Prompter: Now that we have gathered the necessary data, let's move on to the Data Analysis Expert.

Data Analysis Expert:
"""
Based on the data gathered, please analyze the following:

1. Compare the energy mix of Germany and China. How has the percentage of renewable energy changed over the last decade for both countries?
2. Compare the major renewable projects in Germany and China. What impact have these projects had on the renewable energy capacity of each country?
3. Compare the changes in carbon emissions in Germany and China as they have adopted more renewable energy.
4. Compare the impact on the job markets in Germany and China as they have adopted more renewable energy.
5. Compare the renewable energy goals of Germany and China for the next 5 years. How do these goals reflect the trends of the last decade?
"""

--------------------------------------------------------------------------------



2024-07-16 22:44:51,642 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-16 22:44:51,652 - INFO - FINAL ANSWER detected, ending conversation



Ai:
FINAL ANSWER:

Based on the data gathered from Wikipedia, here's a comparison of the economic impact of renewable energy adoption in Germany and China over the last decade:

1. Energy Mix:
   - Germany: The share of renewable electricity rose from just 3.4% of gross electricity consumption in 1990 to exceed 10% by 2005, reaching 42.1% of consumption in 2019. The main types of renewable energy used are wind, biomass, solar, and hydro.
   - China: China's total renewable energy capacity exceeded 1,000 GW in 2021, accounting for 43.5% of the country's total power generation capacity, 10.2 percentage points higher than in 2015. The main types of renewable energy used are hydro, solar, and wind power.

2. Major Renewable Projects:
   - Germany: Germany had the world's largest photovoltaic installed capacity until 2014, and as of 2023 it has over 82 GW. It is also the world's third country by installed total wind power capacity, 64 GW in 2021 (59 GW in 2018) and second for offshore wind